In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Data dir
raw_data_dir = "../data/mimic/raw/"
processed_data_dir = "../data/mimic/processed/"

# Read csv files
pat = pd.read_csv(raw_data_dir + "patients.csv")
adm = pd.read_csv(raw_data_dir + "admissions.csv")
proc = pd.read_csv(raw_data_dir + "procedures_icd.csv")
diag = pd.read_csv(raw_data_dir + "diagnoses_icd.csv")
ie = pd.read_csv(raw_data_dir + "icustays.csv")
d_icd_proc = pd.read_csv(raw_data_dir + "d_icd_procedures.csv")
d_icd_diag = pd.read_csv(raw_data_dir + "d_icd_diagnoses.csv")

processed_mimic = pd.read_csv(processed_data_dir + "sequential_mimic.csv")

/share/apps/anaconda-ci/fgci-centos7-anaconda/software/anaconda/2021-03-tf2/e54ecce8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
adm["admittime"] = pd.to_datetime(adm["admittime"])
adm["dischtime"] = pd.to_datetime(adm["dischtime"])

diag['icd_version'] = diag['icd_version'].astype(int)
proc['icd_version'] = proc['icd_version'].astype(int)

# Inner join to get demographics
demo = pd.merge(pd.merge(pat,adm,on='subject_id'),ie,on=['subject_id', 'hadm_id'])

# Assign age, all patients are older than 18 years old
demo = demo.assign(admission_age = demo['admittime'].dt.year - demo['anchor_year'] + demo['anchor_age'])

# Merge with procedure and diagnosis
demo = pd.merge(demo, diag, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "diag_icd_code", "icd_version": "diag_icd_version", "seq_num": "diag_seq_num"})
demo = pd.merge(demo, proc, on=['subject_id', 'hadm_id'])
demo = demo.rename(columns={"icd_code": "proc_icd_code", "icd_version": "proc_icd_version", "seq_num": "proc_seq_num"})

# Group into higher order categories
def group_and_prefix_diag(code,version):
    return str(version) + "_" + code[:3]

def group_and_prefix_proc(code,version):
    return "IP_" + str(version) + "_" + code

demo['diag_icd_code'] = demo.apply(lambda x: group_and_prefix_diag(x.diag_icd_code, x.diag_icd_version), axis = 1)
demo['proc_icd_code'] = demo.apply(lambda x: group_and_prefix_proc(x.proc_icd_code, x.proc_icd_version), axis = 1)
demo

TypeError: can only concatenate str (not "int") to str

In [1]:
# Get LOS, remove outliers
demo_filter = demo[demo['los'] < 51]
los = demo_filter["los"].unique()

# Seperate into short/stay (long: los > 10 days)
demo_filter = demo_filter.assign(long_los = demo['los'] > 10)

# Filter out duplicates procedures per visit
demo_proc = demo_filter[['hadm_id', 'proc_icd_code', 'long_los']].drop_duplicates()
demo_diag = demo_filter[['hadm_id', 'diag_icd_code', 'long_los']].drop_duplicates()

NameError: name 'demo' is not defined

In [ ]:
print("Short los top 10 proc: \n", demo_proc[demo_proc['long_los'] == False]['proc_icd_code'].value_counts()[:10])
print("Long los top 10 proc: \n", demo_proc[demo_proc['long_los'] == True]['proc_icd_code'].value_counts()[:10])
print("Top 10 proc: \n", demo_proc['proc_icd_code'].value_counts()[:10])

print("Short los top 10 diag: \n", demo_diag[demo_diag['long_los'] == False]['diag_icd_code'].value_counts()[:10])
print("Long los top 10 diag: \n", demo_diag[demo_diag['long_los'] == True]['diag_icd_code'].value_counts()[:10])
print("Top 10 diag: \n", demo_diag['diag_icd_code'].value_counts()[:10])

In [ ]:
print(d_icd_proc[d_icd_proc['icd_code'].str.contains('3893')])
#d_icd_diag[d_icd_diag['icd_code'].str[:3] == '401']

In [ ]:
# Group codes into sequence
group_diag = demo.groupby(['subject_id','hadm_id'])['diag_icd_code'].apply(list)
group_proc = demo.groupby(['subject_id','hadm_id'])['proc_icd_code'].apply(list)
group_age = demo.groupby(['subject_id','hadm_id'])['admission_age'].apply(lambda x: list(x)[0])
group_gender = demo.groupby(['subject_id','hadm_id'])['gender'].apply(lambda x: list(x)[0])
group_ethnicity = demo.groupby(['subject_id','hadm_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_los = demo.groupby(['subject_id','hadm_id'])['los'].apply(lambda x: list(x)[0])

temp_final = pd.concat([group_age, group_gender,group_ethnicity, group_los, group_diag, group_proc], axis=1).reset_index()

In [ ]:
# Group codes into sequence
group_diag_0 = temp_final.groupby(['subject_id'])['diag_icd_code'].apply(list)
group_proc_0 = temp_final.groupby(['subject_id'])['proc_icd_code'].apply(list)
group_age_0 = temp_final.groupby(['subject_id'])['admission_age'].apply(list)
group_gender_0 = temp_final.groupby(['subject_id'])['gender'].apply(lambda x: list(x)[0])
group_ethnicity_0 = temp_final.groupby(['subject_id'])['ethnicity'].apply(lambda x: list(x)[0])
group_los_0 = temp_final.groupby(['subject_id'])['los'].apply(list)
group_hadm_id_0 = temp_final.groupby(['subject_id'])['hadm_id'].apply(list)

final = pd.concat([group_hadm_id_0, group_age_0, group_gender_0 ,group_ethnicity_0 , group_los_0, group_diag_0, group_proc_0], axis=1).reset_index()

In [ ]:
def _sequentialize_codes(diag, proc):

    lst_len = len(diag)
    seqs = []
        
    for i in range(lst_len):
        
        # Filter unique values
        u_diag = list(set(diag[i]))
        u_proc = list(set(proc[i]))
        concat = u_diag + u_proc
        inner_seq = "<SEP>".join(concat)
        seqs.append(inner_seq)
    seq = ";".join(seqs)
    return seq

final["seqential_code"] = final.apply(lambda x: _sequentialize_codes(x.diag_icd_code, x.proc_icd_code), axis = 1)

In [ ]:
# Assign treatment. Treatment is true if patient received 'IP_9_3893' procedure
def assign_treatment(proc):

    lst_len = len(proc)
    seqs = []
        
    for i in range(lst_len):
        if 'IP_9_3893' in proc[i]:
            return 1
    return 0

final["treated"] = final.apply(lambda x: assign_treatment(x.proc_icd_code), axis = 1)

In [ ]:
final.to_csv(data_dir + "sequential_mimic.csv", index=False)

In [41]:
final["treated"].value_counts()

0    10738
1     1727
Name: treated, dtype: int64